In [2]:
import requests
from requests import Response
def get_response() -> Response | None:
    aqi_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=CSV'
    try:
        response:Response = requests.get(aqi_url)
        response.raise_for_status()
        if response.status_code == 200: #有.ok 跟 .status_code 這兩種方式判定
            print("下載成功")
            return response
        else:
            print("下載失敗")
            return None
    except Exception as e: #Exception是檢查錯誤的涵義，然後回傳至設定的e當中
        print(e)
        print("連線失敗")
        return None

In [ ]:
response:Response | None = get_response()
if response:
    rawString:str = response.text
from io import StringIO
import csv
from csv import DictReader
file:StringIO = StringIO(rawString) #轉換成StringIO的實體檔案
reader:DictReader = csv.DictReader(file)
data:list = list(reader)
for row in data:
    print(row)

In [7]:
#由上方產生的結果抓取所需的欄位資料
from pydantic import BaseModel,Field,RootModel,field_validator

class Site(BaseModel):
    站點名稱:str = Field(alias='sitename')
    縣市:str = Field(alias='county')
    aqi:float
    時間:str = Field(alias='datacreationdate')
    pm25:float = Field(alias='pm2.5')

    @field_validator("pm25",mode='before')
    #進行驗證pm25,運用field_vailidator內的mode做轉換前驗證，驗證完則自動把原先資料pm2.5的字串轉換為float
    @classmethod
    def empty_to_zero(cls, value):  #value資料由pm25的資料而來
        if value == '':
            return '0.0'
        else:
            return value
#進行欄位過濾選擇

class Sites(RootModel):
    root:list[Site]
    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, item):
        return self.root(item)
#Root功能回朔資料

sites:Sites = Sites.model_validate(data) #資料驗證後回傳並紀錄

#for item in sites:
#    print(item)
#原本用 for...in...進行資料一筆一筆轉換成python資料形式

In [ ]:
from pprint import pprint
download_data = sites.model_dump() #model_dump()為直接整筆資料轉換為python資料非一筆一筆轉換
pprint(download_data)

In [10]:
import pandas as pd #外部套件，用於轉換顯示的資料方式
pd.DataFrame(download_data) #使用python轉換後的資料來源，sites則是尚未轉換成python版的資料來源

,站點名稱,縣市,aqi,時間,pm25
0,屏東(枋山),屏東縣,36.0,2024-03-28 19:00,7.0
1,新北(樹林),新北市,68.0,2024-03-28 19:00,18.0
2,屏東(琉球),屏東縣,42.0,2024-03-28 19:00,9.0
3,臺南(麻豆),臺南市,61.0,2024-03-28 19:00,13.0
4,高雄(湖內),高雄市,46.0,2024-03-28 19:00,12.0
...,...,...,...,...,...
995,萬華,臺北市,41.0,2024-03-28 08:00,12.0
996,中山,臺北市,37.0,2024-03-28 08:00,12.0
997,士林,臺北市,33.0,2024-03-28 08:00,16.0
998,淡水,新北市,40.0,2024-03-28 08:00,14.0
